# Context Manager

- 사전 조건과 사후 조건이 있는 일부 코드를 실행해야 하는 경우 컨텍스트 관리자를 이용할 수 있다.
- 일반적으로 리소스 관리와 관령하여 컨텍스트 관리자를 자주 볼 수 있다.
- 일반적으로 할당된 모든 리소스를 해제해야하는 경우 사용한다.
- 예외가 발생하거나 오류를 처리해야하는 경우 일반적으로 `finally` 블록에 정리 코드를 넣을 수 있다.

In [1]:
filename = 'test.txt'
fd = open(filename)

def process_file(file: str):
    print(f'{str} processing...')

try:
    process_file(fd)
finally: 
    fd.close()

FileNotFoundError: [Errno 2] No such file or directory: 'test.txt'

위 코드를 컨텍스트 관리자를 통해 파이썬스럽게 구현할 수 있다.

In [ ]:
with open(filename) as fd:
    process_file(fd)

`with`문은 컨텍스트 관리자 프로토콜을 구현한다. 즉 예외가 발생한 경우에도 블록이 완료되면 파일이 자동으로 닫힌다.
- 컨텍스트 관리자는 아래 두개의 매직 메소드로 구성된다.
    - `__enter__`
    - `__exit__`
- 스크립트를 사용해 데이터베이스 백업을 하려는 경우를 가정
    - 백업은 오프라인 상태에서 해야하며 서비스를 잠시 중단해야한다.
    - 백업이 끝나면 성공적으로 진행되었는지에 관계없이 프로세스를 다시 시작한다.


In [ ]:
# 서비스를 중지하고 백업 후 예외 및 특이사항 처리, 서비스 재시작

def stop_database():
    run("systemctl stop postgresql.service")

def start_database():
    run("systemctl start postgresql.service")

class DBHandler:
    # 1. 서비스 중지
    def __enter__(self):
        stop_database()
        return self # 쓸모가 없지만 일반적으로 __enter__ 에서 무언가를 반환하는 것은 좋은 습관
    
    # 블록에서 발생한 예외를 파라미터로 받음
    # __exit__가 True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전달하지 않는다. (주의)
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

# db 백업
def db_backup():
    run("pg_dump database")

def main():
    with DBHandler():
        db_backup()

## contextlib

`__enter__`와 `__exit__` 매직 메소드를 이용하면 컨텍스트 관리자 프로토콜을 지원할 수 있지만 `contextlib` 모듈을 이용하면 보다 쉽게 구현할 수 있다.
- contextmanager 데코레이터를 적용하면 해당 함수의 코드를 컨텍스트 관리자로 변환한다.
- 함수는 제너레이터 형태여야하며 코드의 문장을 __enter__, __exit__로 분리한다.

In [ ]:
import contextlib

@contextlib.contextmanager
def db_handler():
    try:
        stop_database()
        yield # yeild 키워드를 이용해 제너레이터 함수 정의
        # yeild 문 앞의 모든 것은 __enter__ 메소드의 일부처럼 취급된다.
        # yeild 문 뒤의 모든 것은 __exit__ 로직으로 볼 수 있다.
    finally:
        start_database()

with db_handler():
    db_backup()

- 컨텍스트 매니저를 작성하면 기존 함수를 리팩토링하기 쉽다.
- 일반적으로 어느 특정 객체에도 속하지 않은 컨텍스트 관리자가 필요한 경우 좋은 방법이다.

## contextlib.ContextDecorator

- 컨텍스트 관리자 안에서 실행될 함수에 데코레이터를 적용하기 위한 로직을 제공하는 믹스인 클래스
    - 믹스인 클래스: 다른 클래스에서 필요한 기능만 섞어서 사용할 수 있도록 메소드만 제공하는 유틸리티 형태의 클래스

In [ ]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(slef, ext_type, ex_value, ex_traceback):
        start_database()

# offline_backup() 함수가 컨텍스트 관리자 안에서 자동으로 실행됨
@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

이 경우 유일한 단점은 완전히 독립적이기 때문에 offline_backup() 함수에서 꼭 필요한 경우에도 데코레이터 객체에 직접 접근할 수 없다는 점이다.

접근이 필요한 경우 아래와 같이 우회하여 데코레이터 객체에 접근할 수 있다

In [ ]:
def offline_backup():
    with dbhandler_decorator() as handler: ...

## contextlib.supress

- 안전하다고 확신하는 경우 예외를 무시하는 기능이다.
- suppress 메소드를 호출하면 로직에서 자체적으로 처리하고 있는 예외임을 명시한다.

In [2]:
import contextlib

# input 데이터 파싱 시 오류가 발생하면 DataConversionException 발생
with contextlib.suppress(DataConversionException):
    parse_data(input_json_or_dict)